In [297]:
#Import biblioteci si module
import pandas as pd
import os

#Import fisiere in dataframe
path = 'C:\\Users\\anlazar\\Full_Archive\\New Files\\olist db\\'

for file in os.listdir(path):
    if file.startswith('olist') and file.endswith('.csv') and 'leads' not in file and 'filtered' not in file:
        globals()[file.replace('olist_', '').replace('.csv', '').replace('_dataset', '') + '_df'] = pd.read_csv(path+file)
        
#Conversie tipuri de date        
for column in orders_df.columns.values[3:]:
    orders_df[column] = pd.to_datetime(orders_df[column])        

for column in orders_df.columns.values[:2]:
    orders_df[column] = orders_df[column].astype('uint32')
    
orders_df['order_status'] = orders_df['order_status'].astype('category')

#Creare coloane noi
orders_df['delivery_days'] = (orders_df['order_delivered_customer_date'] - orders_df['order_purchase_timestamp']).dt.days
orders_df['estimated_delivery_days'] = (orders_df['order_estimated_delivery_date'] - orders_df['order_purchase_timestamp']).dt.days
orders_df['days_btw_purch_and_approval'] = (orders_df['order_approved_at'] - orders_df['order_purchase_timestamp']).dt.days
orders_df['days_btw_approval_and_carrier'] = (orders_df['order_delivered_carrier_date'] - orders_df['order_approved_at']).dt.days
orders_df['days_btw_carrier_and_customer'] = (orders_df['order_delivered_customer_date'] - orders_df['order_delivered_carrier_date']).dt.days

#Eliminare anomalii
orders_df_w_errors = orders_df[(orders_df['days_btw_approval_and_carrier'] <0) | (orders_df['days_btw_carrier_and_customer'] < 0)]
orders_df = orders_df.drop(orders_df[(orders_df['days_btw_approval_and_carrier'] <0) | (orders_df['days_btw_carrier_and_customer'] < 0)].index)

# Masuri de tendinta centrala: medie, mediana, mod
(
orders_df
    .iloc[:,-5:]
    .mean()
    .to_frame()
    .rename(columns = {0:'mean_val'})
    .merge(
           right = orders_df
                    .iloc[:,-5:]
                    .median()
                    .to_frame()
                    .rename(columns = {0:'median_val'})
           , left_index = True 
           , right_index = True
          )
    .merge(
           right = orders_df
                    .iloc[:, -5:]
                    .mode()
                    .stack()
                    .to_frame()
                    .droplevel(0)
                    .rename(columns = {0:'mode_val'})
           , left_index = True
           , right_index = True
          )
# Masuri de dispersie: gama, gama inter-cvartilica, varianta, deviatie standard
    .merge(
           right = orders_df
                    .iloc[:,-5:]
                    .max()
                    .to_frame()
                    .rename(columns = {0:'max_val'})
           , left_index = True 
           , right_index = True
          )
    .merge(
           right = orders_df
                    .iloc[:,-5:]
                    .min()
                    .to_frame()
                    .rename(columns = {0:'min_val'})
           , left_index = True 
           , right_index = True
          )
    .merge( 
           right = orders_df
                    .iloc[:,-5:]
                    .max()
                    .sub(
                         orders_df
                            .iloc[:,-5:]
                            .min()
                            )
                    .to_frame()
                    .rename(columns = {0:'range_val'})
           , left_index = True 
           , right_index = True
          )
    .merge( 
            right = orders_df
                    .iloc[:,-5:]
                    .quantile([0.25, 0.75])
                    #.to_frame()
                    .unstack()
                    .unstack()
                    .assign(iqr = lambda df: df[0.75] - df[0.25])
                    .assign(upper_whisker = lambda df: df[0.75] + 1.5 * df['iqr'])
                    #['iqr']
           , left_index = True 
           , right_index = True
            )
)



,mean_val,median_val,mode_val,max_val,min_val,range_val,0.25,0.75,iqr,upper_whisker
delivery_days,12.154200,10.0,7.0,209.0,0.0,209.0,6.0,15.0,9.0,28.5
estimated_delivery_days,23.414563,23.0,21.0,155.0,1.0,154.0,18.0,28.0,10.0,43.0
days_btw_purch_and_approval,0.247916,0.0,0.0,187.0,0.0,187.0,0.0,0.0,0.0,0.0
days_btw_approval_and_carrier,2.357843,1.0,0.0,125.0,0.0,125.0,0.0,3.0,3.0,7.5
days_btw_carrier_and_customer,8.911065,7.0,6.0,205.0,0.0,205.0,4.0,12.0,8.0,24.0


In [294]:
(orders_df
.iloc[:,-5:]
.quantile([0.25, 0.75])
#.to_frame()
.unstack()
.unstack()
.assign(iqr = lambda df: df[0.75] - df[0.25])
#['iqr']
)

,0.25,0.75,iqr
delivery_days,6.0,15.0,9.0
estimated_delivery_days,18.0,28.0,10.0
days_btw_purch_and_approval,0.0,0.0,0.0
days_btw_approval_and_carrier,0.0,3.0,3.0
days_btw_carrier_and_customer,4.0,12.0,8.0


In [299]:
(
orders_df
[orders_df['delivery_days'] > 28.5]
[['order_status', 'order_purchase_timestamp', 'order_delivered_customer_date', 'delivery_days']]
.sort_values(by = 'delivery_days', ascending = False)
)

,order_status,order_purchase_timestamp,order_delivered_customer_date,delivery_days
2535,delivered,2017-02-21 23:31:00,2017-09-19 14:36:00,209.0
57992,delivered,2018-02-23 14:57:00,2018-09-19 23:24:00,208.0
3494,delivered,2017-03-07 23:59:00,2017-09-19 15:12:00,195.0
3547,delivered,2017-03-08 18:09:00,2017-09-19 14:33:00,194.0
3601,delivered,2017-03-09 13:26:00,2017-09-19 14:38:00,194.0
3568,delivered,2017-03-08 22:47:00,2017-09-19 14:00:00,194.0
45732,delivered,2018-01-03 09:44:00,2018-07-13 20:51:00,191.0
3974,delivered,2017-03-13 20:17:00,2017-09-19 17:00:00,189.0
4124,delivered,2017-03-15 11:24:00,2017-09-19 14:38:00,188.0
4204,delivered,2017-03-15 23:23:00,2017-09-19 17:14:00,187.0


In [303]:
for column in order_reviews_df.columns.values[-2:]:
    order_reviews_df[column] = pd.to_datetime(order_reviews_df[column])

In [319]:
(
orders_df
[orders_df['delivery_days'] > 28.5]
[['order_id', 'order_status', 'order_purchase_timestamp', 'order_estimated_delivery_date', 'order_delivered_customer_date', 
  'delivery_days', 'estimated_delivery_days']]
.merge(right = order_reviews_df[['order_id', 'review_score', 'review_answer_timestamp']])
.assign(
        review_delta_delivery = lambda df: (df['review_answer_timestamp'] - df['order_delivered_customer_date']).dt.days,
        review_delta_estimated = lambda df: (df['review_answer_timestamp'] - df['order_estimated_delivery_date']).dt.days,
        delivered_delta_estimated = lambda df: df['delivery_days'] - df['estimated_delivery_days']
       )
#.sort_values(by = 'review_delta_delivery')
#.sort_values(by = 'review_delta_estimated')
.sort_values(by = 'delivered_delta_estimated', ascending = False)
)

,order_id,order_status,order_purchase_timestamp,order_estimated_delivery_date,order_delivered_customer_date,delivery_days,estimated_delivery_days,review_score,review_answer_timestamp,review_delta_delivery,review_delta_estimated,delivered_delta_estimated
3162,57993,delivered,2018-02-23 14:57:00,2018-03-15,2018-09-19 23:24:00,208.0,19,2,2018-03-17 05:08:00,-187,2,189.0
141,2536,delivered,2017-02-21 23:31:00,2017-03-22,2017-09-19 14:36:00,209.0,28,1,2017-03-26 22:57:00,-177,4,181.0
2146,45733,delivered,2018-01-03 09:44:00,2018-01-19,2018-07-13 20:51:00,191.0,15,1,2018-01-24 16:45:00,-171,5,176.0
192,3975,delivered,2017-03-13 20:17:00,2017-04-05,2017-09-19 17:00:00,189.0,22,1,2017-04-09 09:56:00,-164,4,167.0
177,3569,delivered,2017-03-08 22:47:00,2017-04-06,2017-09-19 14:00:00,194.0,28,1,2017-04-08 17:35:00,-164,2,166.0
173,3495,delivered,2017-03-07 23:59:00,2017-04-07,2017-09-19 15:12:00,195.0,30,1,2017-04-13 12:34:00,-160,6,165.0
179,3602,delivered,2017-03-09 13:26:00,2017-04-11,2017-09-19 14:38:00,194.0,32,4,2017-03-29 21:10:00,-174,-13,162.0
509,13031,delivered,2017-06-12 13:14:00,2017-06-26,2017-12-04 18:36:00,175.0,13,4,2017-07-04 11:33:00,-154,8,162.0
198,4205,delivered,2017-03-15 23:23:00,2017-04-10,2017-09-19 17:14:00,187.0,25,3,2017-04-06 15:30:00,-167,-4,162.0
196,4125,delivered,2017-03-15 11:24:00,2017-04-13,2017-09-19 14:38:00,188.0,28,3,2017-04-16 01:46:00,-157,3,160.0
